In [1]:
import json
from textblob import TextBlob
import pandas as pd
import json
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from textblob import TextBlob
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm


/Users/yonghui/opt/anaconda3/envs/py310/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re


model_name = "cardiffnlp/twitter-roberta-base-irony"
sarcasm_tokenizer = AutoTokenizer.from_pretrained(model_name)
sarcasm_model = AutoModelForSequenceClassification.from_pretrained(model_name)

def preprocess_text(text: str) -> str:
    text = re.sub(r"http\S+", "http", text)  
    text = re.sub(r"@\w+", "@user", text)    
    return text

def detect_sarcasm(text: str) -> bool:
    """Returns True if text is sarcastic, False otherwise."""
    text = preprocess_text(text)  


    inputs = sarcasm_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)


    with torch.no_grad():
        logits = sarcasm_model(**inputs).logits
    probs = torch.softmax(logits, dim=1).squeeze()

    
    return probs[1].item() > 0.5  



In [ ]:
def sentiment_only(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return "Positive"
    elif analysis.sentiment.polarity < 0:
        return "Negative"
    return "Neutral"

def polarity_then_sentiment(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    subjectivity = analysis.sentiment.subjectivity

    
    if polarity > 0:
        sentiment_direction = "Positive"
    elif polarity < 0:
        sentiment_direction = "Negative"
    else:
        sentiment_direction = "Neutral"

    
    if sentiment_direction == "Positive":
        pos_threshold = 0.1 if subjectivity > 0.5 else 0.2
        return "Positive" if polarity > pos_threshold else "Neutral"
    
    elif sentiment_direction == "Negative":
        neg_threshold = -0.1 if subjectivity > 0.5 else -0.2
        return "Negative" if polarity < neg_threshold else "Neutral"
    
    else:
        return "Neutral"

In [ ]:
def enhanced_sentiment(text: str) -> str:
  
    polarity = TextBlob(text).sentiment.polarity
    is_sarcastic = detect_sarcasm(text)

   
    if is_sarcastic:
        polarity = -polarity  

    if polarity > 0.2:
        return "Positive"
    elif polarity < -0.2:
        return "Negative"
    else:
        return "Neutral"


In [ ]:
def evaluate_methods(dataset):
    methods = {
        "Sentiment Only": sentiment_only,
        "Polarity then Sentiment": polarity_then_sentiment,
        "Enhanced Sentiment (with Sarcasm)": enhanced_sentiment
    }

    results = {
        method: {
            "y_true": [],
            "y_pred": []
        }
        for method in methods
    }

    for entry in dataset:
        text = entry["text"]
        
        ann1 = entry["manual_labels"]["annotator1"]["polarity"]
        ann2 = entry["manual_labels"]["annotator2"]["polarity"]
        ground_truth = ann1 if ann1 == ann2 else "Neutral"

        for method_name, method_func in methods.items():
            pred = method_func(text)
            results[method_name]["y_true"].append(ground_truth)
            results[method_name]["y_pred"].append(pred)

    for method_name in methods:
        y_true = results[method_name]["y_true"]
        y_pred = results[method_name]["y_pred"]

        acc = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_true, y_pred, average="macro", zero_division=0
        )

        print(f"\n=== {method_name} ===")
        print(f"Accuracy:  {acc:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall:    {recall:.4f}")
        print(f"F1 Score:  {f1:.4f}")

with open("evaluationDataset.json", "r") as f:
    dataset = json.load(f)

evaluate_methods(dataset)


=== Sentiment Only ===
Accuracy:  0.3333
Precision: 0.4000
Recall:    0.5000
F1 Score:  0.3333

=== Polarity then Sentiment ===
Accuracy:  0.3333
Precision: 0.2500
Recall:    0.5000
F1 Score:  0.3000

=== Enhanced Sentiment (with Sarcasm) ===
Accuracy:  0.8333
Precision: 0.8889
Recall:    0.8889
F1 Score:  0.8667
